<a href="https://colab.research.google.com/github/JackeYou/Enjoy_LLM/blob/main/infer_local_gpu_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# gpu 本地部署
!git clone https://huggingface.co/THUDM/chatglm-6b

Cloning into 'chatglm-6b'...
remote: Enumerating objects: 350, done.
remote: Counting objects: 100% (177/177), done.
remote: Compressing objects: 100% (177/177), done.
remote: Total 350 (delta 106), reused 0 (delta 0), pack-reused 173
Receiving objects: 100% (350/350), 136.80 KiB | 663.00 KiB/s, done.
Resolving deltas: 100% (205/205), done.
Filtering content: 100% (9/9), 12.49 GiB | 87.95 MiB/s, done.


In [2]:
%pwd
!git clone https://github.com/JackeYou/ChatGLM-Tuning.git
%cd  ChatGLM-Tuning
!pip install -r requirements.txt

Cloning into 'ChatGLM-Tuning'...
remote: Enumerating objects: 146, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 146 (delta 46), reused 33 (delta 33), pack-reused 70
Receiving objects: 100% (146/146), 8.02 MiB | 17.48 MiB/s, done.
Resolving deltas: 100% (76/76), done.
/content/ChatGLM-Tuning
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-02rs7oda
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-02rs7oda
  Resolved https://github.com/huggingface/peft.git to commit 445940fb7b5d38390ffb6707e2a989e89fff03b5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [3]:
%cd  /content/chatglm-6b

/content/chatglm-6b


In [4]:
from modeling_chatglm import ChatGLMForConditionalGeneration
import torch
from peft import PeftModel
from tokenization_chatglm import ChatGLMTokenizer
from transformers import AutoTokenizer

# 加载本地预训练模型
torch.set_default_tensor_type(torch.cuda.HalfTensor)
BASE_MODEL_DIR = "/content/chatglm-6b"
gpu_model = ChatGLMForConditionalGeneration.from_pretrained(BASE_MODEL_DIR).half().cuda()
tokenizer = ChatGLMTokenizer.from_pretrained(BASE_MODEL_DIR)



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//172.28.0.1'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_save_url=https'), Po

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [6]:
%cp /content/ChatGLM-Tuning/cover_alpaca2jsonl.py /content/chatglm-6b/

In [7]:
from cover_alpaca2jsonl import format_example

instructions = [
 {'instruction': '请给出十字路口交通指示灯的三种颜色',
  'input': '',
  'output': '红色、绿色、黄色。',
 },
 {'instruction': '请给出以下资讯3个关键词',
  'input': '猪肉概念股大幅拉升，巨星农牧涨停，新五丰连续两日大涨',
  'output': '猪肉概念股，巨星农牧，新五丰。',
 },
 {'instruction': '以下哪家公司市值最高',
  'input': '腾讯、京东、淘宝',
  'output': '腾讯公司市值最高。截止2021年4月21日，腾讯公司的市值约为6.78万亿人民币，京东的市值约为1.26万亿人民币，淘宝的市值约为1.8万亿人民币。',
 },
 {'instruction': '请给出以下资讯3个关键词',
  'input': '隆基绿能董事长：光伏产品短期价格涨跌意义不大，长期看成本一定会越来越低',
  'output': '1.隆基绿能，2.光伏产品，3.成本降低',
 }
]
with torch.no_grad():
    for idx, item in enumerate(instructions):
        feature = format_example(item)
        input_text = feature['context']
        ids = tokenizer.encode(input_text)
        input_ids = torch.tensor([ids])
        out = gpu_model.generate(
            input_ids=input_ids,
            max_length=150,
            do_sample=False,
            temperature=0
        )
        out_text = tokenizer.decode(out[0])
        answer = out_text.replace(input_text, "").replace("\nEND", "").strip()
        item['infer_answer'] = answer
        print(out_text)
        print(f"chatGPT### {idx+1}.Answer:\n", item.get('output'), '\n\n')

Instruction: 请给出十字路口交通指示灯的三种颜色
Answer: 十字路口交通指示灯通常有红色、绿色和蓝色三种颜色。
chatGPT### 1.Answer:
 红色、绿色、黄色。 


Instruction: 请给出以下资讯3个关键词
Input: 猪肉概念股大幅拉升,巨星农牧涨停,新五丰连续两日大涨
Answer: 猪肉,巨星农牧,新五丰
chatGPT### 2.Answer:
 猪肉概念股，巨星农牧，新五丰。 


Instruction: 以下哪家公司市值最高
Input: 腾讯、京东、淘宝
Answer: 腾讯市值最高。
chatGPT### 3.Answer:
 腾讯公司市值最高。截止2021年4月21日，腾讯公司的市值约为6.78万亿人民币，京东的市值约为1.26万亿人民币，淘宝的市值约为1.8万亿人民币。 


Instruction: 请给出以下资讯3个关键词
Input: 隆基绿能董事长:光伏产品短期价格涨跌意义不大,长期看成本一定会越来越低
Answer: 光伏产品,价格,董事长,隆基绿能。
chatGPT### 4.Answer:
 1.隆基绿能，2.光伏产品，3.成本降低 




In [14]:
import os
pid = os.getpid()
print(pid)
!kill -9 $pid

2602